### Load data

In [103]:
CH_HOST = 'http://localhost:8123' # default address 
import requests
import pandas as pd
import tqdm

def get_clickhouse_data(query, host = CH_HOST, connection_timeout = 1500):
  r = requests.post(host, params = {'query': query}, 
    timeout = connection_timeout)
  if r.status_code == 200:
      return r.text
  else: 
      raise ValueError('Database returned the following error:n' + r.text)

In [49]:
df = pd.read_csv('./data/flight_data_2024.csv')

/var/folders/7v/1ln722x97kd8bchgxpmdkynw0000gn/T/ipykernel_61779/2892042302.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/flight_data_2024.csv')


In [50]:
# Create table SQL query
create_table_query = """
CREATE or replace TABLE flight_data (
    year UInt16,
    month UInt8,
    day_of_month UInt8,
    day_of_week UInt8,
    fl_date Date,
    op_unique_carrier String,
    op_carrier_fl_num Nullable(Float64),
    origin String,
    origin_city_name String,
    origin_state_nm String,
    dest String,
    dest_city_name String,
    dest_state_nm String,
    crs_dep_time Nullable(Float64),
    dep_time Nullable(Float64),
    dep_delay Nullable(Float64),
    taxi_out Nullable(Float64),
    wheels_off Nullable(Float64),
    wheels_on Nullable(Float64),
    taxi_in Nullable(Float64),
    crs_arr_time Nullable(Float64),
    arr_time Nullable(Float64),
    arr_delay Nullable(Float64),
    cancelled UInt8,
    cancellation_code String,
    diverted UInt8,
    crs_elapsed_time Nullable(Float64),
    actual_elapsed_time Nullable(Float64),
    air_time Nullable(Float64),
    distance Nullable(Float64),
    carrier_delay Nullable(Float64),
    weather_delay Nullable(Float64),
    nas_delay Nullable(Float64),
    security_delay Nullable(Float64),
    late_aircraft_delay Nullable(Float64)
) ENGINE = MergeTree()
ORDER BY (fl_date, origin, dest)
"""

# Create the table
result = get_clickhouse_data(create_table_query)
print("Table creation result:", result)

Table creation result: 


In [51]:
# Insert data into ClickHouse
# Replace NaN values with None for proper handling in ClickHouse
df_clean = df.copy()

# Convert fl_date to proper date format
df_clean['fl_date'] = pd.to_datetime(df_clean['fl_date'])

# Fill NaN values with None (will be NULL in ClickHouse)
df_clean = df_clean.where(pd.notnull(df_clean), None)

# Insert data in batches
batch_size = 10000
total_rows = len(df_clean)

for i in tqdm.tqdm(range(0, total_rows, batch_size)):
    batch = df_clean.iloc[i:i+batch_size]
    
    # Convert batch to CSV format for insertion
    csv_data = batch.to_csv(index=False, header=False)
    
    # Insert using ClickHouse CSV format
    insert_query = "INSERT INTO flight_data FORMAT CSV"
    
    try:
        r = requests.post(
            CH_HOST,
            params={'query': insert_query},
            data=csv_data.encode('utf-8'),
            timeout=1500
        )
        
        if r.status_code != 200:
            print(f"Error at batch {i}: {r.text}")
            break
    except Exception as e:
        print(f"Exception at batch {i}: {str(e)}")
        break

print(f"Data insertion completed")

100%|██████████| 708/708 [01:17<00:00,  9.18it/s]

Data insertion completed


In [52]:
# Verify data insertion
# Check row count
count_query = "SELECT COUNT(*) as count FROM flight_data"
count_result = get_clickhouse_data(count_query)
print(f"Total rows in ClickHouse: {count_result}")
print(f"Total rows in DataFrame: {len(df)}")

# Check sample data
sample_query = "SELECT * FROM flight_data LIMIT 5"
sample_result = get_clickhouse_data(sample_query)
print("\nSample data from ClickHouse:")
print(sample_result)

Total rows in ClickHouse: 7079081

Total rows in DataFrame: 7079081

Sample data from ClickHouse:
2024	1	1	1	2024-01-01	9E	5251	ABE	Allentown/Bethlehem/Easton, PA	Pennsylvania	ATL	Atlanta, GA	Georgia	600	558	-2	9	607	749	8	832	757	-35	0		0	152	119	102	692	0	0	0	0	0
2024	1	1	1	2024-01-01	9E	5381	ABE	Allentown/Bethlehem/Easton, PA	Pennsylvania	ATL	Atlanta, GA	Georgia	1231	1240	9	13	1253	1432	5	1450	1437	-13	0		0	139	117	99	692	0	0	0	0	0
2024	1	1	1	2024-01-01	OH	5052	ABE	Allentown/Bethlehem/Easton, PA	Pennsylvania	CLT	Charlotte, NC	North Carolina	1713	1700	-13	11	1711	1824	15	1912	1839	-33	0		0	119	99	73	481	0	0	0	0	0
2024	1	1	1	2024-01-01	OH	5156	ABE	Allentown/Bethlehem/Easton, PA	Pennsylvania	CLT	Charlotte, NC	North Carolina	1158	1442	164	15	1457	1614	6	1357	1620	143	0		0	119	98	77	481	14	0	0	0	129
2024	1	1	1	2024-01-01	OH	5600	ABE	Allentown/Bethlehem/Easton, PA	Pennsylvania	CLT	Charlotte, NC	North Carolina	600	558	-2	31	629	747	11	803	758	-5	0		0	123	120	78	481	0	0	0	0	0



### Generate Q&A Pairs for ClickHouse SQL

In [68]:
df.sample(5).T

,6651582,5320012,90208,3867145,5722329
year,2024,2024,2024,2024,2024
month,12,10,1,7,10
day_of_month,9,2,5,20,22
day_of_week,1,3,5,6,2
fl_date,2024-12-09,2024-10-02,2024-01-05,2024-07-20,2024-10-22
op_unique_carrier,MQ,DL,WN,YX,AA
op_carrier_fl_num,4255.0,1023.0,863.0,4321.0,2506.0
origin,DFW,PVD,IND,DCA,TPA
origin_city_name,"Dallas/Fort Worth, TX","Providence, RI","Indianapolis, IN","Washington, DC","Tampa, FL"
origin_state_nm,Texas,Rhode Island,Indiana,Virginia,Florida


In [84]:
df.groupby(['origin', 'dest']).agg(
    {'fl_date': 'count', 'arr_delay': 'mean', 'dep_delay': 'mean',
     'distance': ['mean', 'std']}
).sort_values(by=('fl_date', 'count'), ascending=False).head(10)

fl_date  arr_delay  dep_delay distance     
              count       mean       mean     mean  std
origin dest                                            
OGG    HNL    11642   0.374370   1.721542    100.0  0.0
HNL    OGG    11634   0.908341   2.269131    100.0  0.0
LAX    SFO    11526  14.929675  14.511947    337.0  0.0
SFO    LAX    11499   8.936632  14.755798    337.0  0.0
LGA    ORD    10971   5.601290  13.192626    733.0  0.0
ORD    LGA    10969  10.426741  15.596860    733.0  0.0
JFK    LAX     9915  -2.945431   9.103815   2475.0  0.0
LAX    JFK     9915   3.281393  13.582194   2475.0  0.0
DCA    BOS     9658   5.819648  12.747792    399.0  0.0
BOS    DCA     9657   4.079641   9.498843    399.0  0.0

In [120]:
import json

qa_pairs = [
    {
        'question': 'How many flights were in 2024?',
        'sql': 'select count() from flight_data where year = 2024 format TabSeparatedWithNames'
    },
    {
        'question': 'From which airport were the most flights departing in July 2024?',
        'sql': "select origin, count() as flight_count from flight_data where toStartOfMonth(fl_date) = '2024-07-01' group by origin order by flight_count desc limit 1 format TabSeparatedWithNames"
    }, 
    {
        'question': 'What route on average took the longest time in 2024?',
        'sql': "select concat(origin, ' - ', dest) as route, avg(actual_elapsed_time) as avg_elapsed_time from flight_data where year = 2024 group by route order by avg_elapsed_time desc limit 1 format TabSeparatedWithNames"
    },
    {
        'question': 'In 2024, what percentage of time all airplanes spent in the air?',
        'sql': "select (sum(air_time) / sum(actual_elapsed_time)) * 100 as percentage_in_air from flight_data format TabSeparatedWithNames"
    },
    {
        'question': 'Which airline had the highest average departure delay in 2024?',
        'sql': "select op_unique_carrier, avg(dep_delay) as avg_dep_delay from flight_data where year = 2024 group by op_unique_carrier order by avg_dep_delay desc limit 1 format TabSeparatedWithNames"
    },
    {
        'question': 'Which airline had the highest rate of cancellations in 2024?',
        'sql': "select op_unique_carrier, (sum(cancelled) / count()) * 100 as cancellation_rate from flight_data where year = 2024 group by op_unique_carrier order by cancellation_rate desc limit 1 format TabSeparatedWithNames"
    }, 
    { 
        'question': 'Are delays different on weekends compared to weekdays?',
        'sql': "select if(day_of_week in (6,7), 'Weekend', 'Weekday') as day_type, avg(arr_delay) as avg_arr_delay from flight_data where year = 2024 group by day_type order by avg_arr_delay desc format TabSeparatedWithNames"
    },
    {
        'question': "I'm flying from LAX to SFO, I supposed to arrive at 12:00, but I have a very important meeting so I must leave airport at 12:20 max. Based on historical data, what is the probability that my flight will be delayed and I won't make it to my meeting on time?",
        'sql': "select (countIf(arr_delay > 20) / count()) * 100 as delay_probability from flight_data where origin = 'LAX' and dest = 'SFO' format TabSeparatedWithNames"
    }, 
    {
        'question': 'Are flights late more often than being early (not taking into account time difference less than 10 mins)?',
        'sql': "select multiIf(arr_delay > 10, 'Late', arr_delay < -10, 'Early', 'On Time') as status, count() as count from flight_data group by status order by count desc format TabSeparatedWithNames"
    },
    {
        'question': 'In case there were delay > 15 mins on departure, what is the probability that arrival would be on time?',
        'sql': "select (countIf(arr_delay <= 0) / count()) * 100 as on_time_arrival_probability from flight_data where dep_delay > 15 format TabSeparatedWithNames"
    },
    {
        'question': 'What was the highest speed in mph?',
        'sql': "select max(distance / (air_time / 60)) as max_speed from flight_data where air_time > 0 format TabSeparatedWithNames"
    }, 
    {
        'question': 'Are there any discrepancies with the distance between two airports in data? Meaning cases when distance is different for the route (we should take into account both A-B and B-A routes).',
        'sql': "select if(origin < dest, concat(origin, '-', dest), concat(dest, '-', origin)) as route, count(distinct distance) as distance_variants from flight_data group by route having distance_variants > 1 order by distance_variants desc format TabSeparatedWithNames"
    },
    {
        'question': 'What are top 5 biggest airlines by number of flights in 2024?',
        'sql': "select op_unique_carrier, count() as flight_count from flight_data where year = 2024 group by op_unique_carrier order by flight_count desc limit 5 format TabSeparatedWithNames"
    },
    {
        'question': 'Are departure delays more common in the morning (6am-12pm) or in the evening (6pm-12am)?',
        'sql': "select multiIf(crs_dep_time >= 600 and crs_dep_time < 1200, 'Morning', crs_dep_time >= 1800 and crs_dep_time < 2400, 'Evening', 'Other') as time_of_day, avg(dep_delay) as avg_dep_delay from flight_data where year = 2024 group by time_of_day order by avg_dep_delay desc format TabSeparatedWithNames"
    }, 
    {
        'question': 'Are weather delays more common in summer or winter?',
        'sql': "select if(month in (6,7,8), 'Summer', 'Winter') as season, avg(weather_delay) as avg_weather_delay from flight_data where year = 2024 and month in (6,7,8,12,1,2) group by season order by avg_weather_delay desc format TabSeparatedWithNames"
    },
    {
        'question': 'Which airline has the fastest planes on average?',
        'sql': "select op_unique_carrier, avg(distance / (air_time / 60)) as avg_speed from flight_data where air_time > 0 group by op_unique_carrier order by avg_speed desc format TabSeparatedWithNames"
    }, 
    { 
        'question': 'Are cancellations more often on Christmas (24-26 December) vs non-holiday days in December?',
        'sql': "select multiIf(fl_date in ('2024-12-24', '2024-12-25', '2024-12-26'), 'Holiday', 'Non-Holiday') as day_type, (sum(cancelled) / count()) * 100 as cancellation_rate from flight_data where year = 2024 and month = 12 group by day_type order by cancellation_rate desc format TabSeparatedWithNames"
    },
    {
        'question': 'Are diversions more common then cancellations?',
        'sql': "select (sum(diverted) / count()) * 100 as diversion_rate, (sum(cancelled) / count()) * 100 as cancellation_rate from flight_data format TabSeparatedWithNames"
    }, 
    {
        'question': 'How often are flights departing early (more than 10 minutes before scheduled time)?',
        'sql': "select (countIf(dep_delay < -10) / count()) * 100 as early_departure_rate from flight_data format TabSeparatedWithNames"
    },
    {
        'question': 'What is the average taxi out time for flights departing from JFK airport?',
        'sql': "select avg(taxi_out) as avg_taxi_out_time from flight_data where origin = 'JFK' and taxi_out is not null format TabSeparatedWithNames"
    }
]

In [121]:
for rec in tqdm.tqdm(qa_pairs): 
    if 'format TabSeparatedWithNames' not in rec['sql']:
        print(
    f'''
========FORMAT========
question: {rec['question']}
SQL: {rec['sql']}
'''
        )
    try: 
        get_clickhouse_data(rec['sql'])
    except:
        print(
f'''
========FAILED========
question: {rec['question']}
SQL: {rec['sql']}
'''
        )

100%|██████████| 20/20 [00:00<00:00, 26.39it/s]


In [122]:
# Save to JSON file
output_file = './data/flight_data_qa_pairs.json'
with open(output_file, 'w') as f:
    json.dump(qa_pairs, f, indent=2)
    
print(f"✓ Saved {len(qa_pairs)} Q&A pairs to {output_file}")

✓ Saved 20 Q&A pairs to ./data/flight_data_qa_pairs.json
